In [1]:
import pandas as pd
import numpy as np
import datetime
import orquest_csv_utils as ocu

utils imported


In [2]:
import os

In [3]:
# Load relevant data
associations = pd.read_csv('associations_rich.csv')
hours = pd.read_csv('hours_rich.csv')
incidences = pd.read_csv('incidences_rich.csv')

'''
 Here, we will perform a simple casting
 of the date columns so that they are
 treated as datetime objects
'''

associations['from_date'] = pd.to_datetime(associations['from_date'])
# If association has no end_date, we will assume current date in order to allow date range comparisons
associations['to_date'] = associations['to_date'].fillna(datetime.datetime.today().date())
associations['to_date'] = pd.to_datetime(associations['to_date'])
hours['day'] = pd.to_datetime(hours['day'])
incidences['from_date'] = pd.to_datetime(incidences['from_date'])
# If incidence has no end_date, we will assume current date in order to allow date range comparisons
incidences['to_date'] = incidences['to_date'].fillna(datetime.datetime.today().date())
incidences['to_date'] = pd.to_datetime(incidences['to_date'])

In [4]:
'''
 This analysis aims to provide a work log
 that informs about the state of workers at
 any given time.
'''

# Join hours and associations to determine whether or not
# the planned hours occured in the worker's original workplace.

hours_associations = pd.merge(hours, associations, how='left',on='person_id')

In [5]:
hours_associations.dtypes

day             datetime64[ns]
hour                     int64
person_id                int64
worked_hours           float64
from_date       datetime64[ns]
to_date         datetime64[ns]
store_id                 int64
dtype: object

In [6]:
# Filter associations to those overlapping with the respective date of the `hours` row. 
# We use >= and <= because both from and to are inclusive.

hours_associations = hours_associations[(hours_associations['day'] >= hours_associations['from_date']) & (hours_associations['day'] <= hours_associations['to_date'])]

In [7]:
'''
 Since associations can overlap due to exceptional circumstances,
 we will associate each day with the association with the latest
 from_date, out of the ones that include 'day' in their range
'''
# Identify latest associations
latest_associations = hours_associations.groupby(['day','hour','person_id'])['from_date'].idxmax()
hours_associations.loc[latest_associations]

,day,hour,person_id,worked_hours,from_date,to_date,store_id
17260,2023-11-01,7,665956,1.0,2023-07-01,2024-04-26,2
17264,2023-11-01,7,1031417,1.0,2022-05-23,2024-04-26,2
17265,2023-11-01,8,665956,1.0,2023-07-01,2024-04-26,2
17269,2023-11-01,8,1031417,1.0,2022-05-23,2024-04-26,2
17270,2023-11-01,9,665926,1.0,2008-10-01,2024-04-26,2
...,...,...,...,...,...,...,...
17251,2024-01-31,20,665959,1.0,2023-07-01,2024-04-26,1
17253,2024-01-31,20,665971,1.0,2008-10-01,2024-04-26,1
17254,2024-01-31,20,665987,1.0,2021-12-01,2024-04-26,1
17256,2024-01-31,20,1305178,1.0,2022-11-09,2024-04-26,1


In [8]:
hours_associations = hours_associations.loc[latest_associations]

In [9]:
# # Bring matches to original Hours table.

hours_associations = pd.merge(hours, hours_associations[['day', 'person_id', 'hour','from_date', 'to_date','store_id']], how='left', on=['day', 'person_id', 'hour'])

In [10]:
hours_associations

,day,hour,person_id,worked_hours,from_date,to_date,store_id
0,2023-11-02,6,665917,1.0,2023-07-01,2024-03-31,1
1,2023-11-02,6,665948,1.0,2008-10-01,2024-04-26,1
2,2023-11-02,6,665954,1.0,2020-10-01,2024-04-26,1
3,2023-11-02,6,665986,1.0,2008-10-01,2024-04-26,1
4,2023-11-02,6,999012,1.0,2022-10-12,2024-04-26,1
...,...,...,...,...,...,...,...
14111,2024-01-20,22,665923,0.5,2008-10-01,2024-04-26,2
14112,2024-01-20,22,665926,0.5,2008-10-01,2024-04-26,2
14113,2024-01-20,22,665979,0.5,2023-07-01,2024-04-26,2
14114,2024-01-20,22,665997,0.5,2023-07-01,2024-04-26,2


In [11]:
# Useful for previewing, commented out because technically not needed
# hours_associations.sort_values(by=['day','person_id','hour'],inplace=True)

In [12]:
hours_associations.rename(columns={'from_date':'asoc_from_date','to_date':'asoc_to_date'},inplace=True)

In [13]:
# hours_associations = hours_associations[['day','person_id','planned_hours','store_id']].sort_values(by=['person_id','day']).drop_duplicates(subset=['day','person_id'])

In [14]:
'''
 At this stage we have the planned allocation per day and workplace.
 We need to cross reference against worker absences to verify that
 they did indeed work.
'''

hours_associations_incidences = pd.merge(hours_associations, incidences, how='left', on='person_id')

In [15]:
hours_associations_incidences.rename(columns={'from_date':'inc_from_date','to_date':'inc_to_date'},inplace=True)
hours_associations_incidences

,day,hour,person_id,worked_hours,asoc_from_date,asoc_to_date,store_id,type_name,inc_from_date,inc_to_date
0,2023-11-02,6,665917,1.0,2023-07-01,2024-03-31,1,Vacaciones,2023-11-20,2023-11-21
1,2023-11-02,6,665917,1.0,2023-07-01,2024-03-31,1,Vacaciones,2023-11-13,2023-11-19
2,2023-11-02,6,665917,1.0,2023-07-01,2024-03-31,1,Licencia sin sueldo,2023-12-30,2023-12-31
3,2023-11-02,6,665948,1.0,2008-10-01,2024-04-26,1,Día de cumpleaños,2024-01-02,2024-01-02
4,2023-11-02,6,665948,1.0,2008-10-01,2024-04-26,1,Vacaciones,2023-11-24,2023-11-24
...,...,...,...,...,...,...,...,...,...,...
30278,2024-01-20,22,665979,0.5,2023-07-01,2024-04-26,2,NaN,NaT,NaT
30279,2024-01-20,22,665997,0.5,2023-07-01,2024-04-26,2,Día de cumpleaños,2024-01-02,2024-01-02
30280,2024-01-20,22,665997,0.5,2023-07-01,2024-04-26,2,Vacaciones,2023-12-04,2023-12-05
30281,2024-01-20,22,665997,0.5,2023-07-01,2024-04-26,2,Enfermedad común,2024-01-12,2024-02-17


In [16]:
# Filter incidences to those overlapping with the respective date of the `hours` row. 
# We use >= and <= because both from and to are inclusive.
hours_associations_incidences = hours_associations_incidences[(hours_associations_incidences['day'] >= hours_associations_incidences['inc_from_date']) & (hours_associations_incidences['day'] < hours_associations_incidences['inc_to_date'])]


In [17]:
# Bring matches to original Hours table
hours_associations_incidences = pd.merge(hours_associations, hours_associations_incidences[['day','hour','person_id','type_name','inc_from_date','inc_to_date']], how='left', on=['day','hour','person_id'])

In [18]:
hours_associations_incidences

,day,hour,person_id,worked_hours,asoc_from_date,asoc_to_date,store_id,type_name,inc_from_date,inc_to_date
0,2023-11-02,6,665917,1.0,2023-07-01,2024-03-31,1,NaN,NaT,NaT
1,2023-11-02,6,665948,1.0,2008-10-01,2024-04-26,1,NaN,NaT,NaT
2,2023-11-02,6,665954,1.0,2020-10-01,2024-04-26,1,NaN,NaT,NaT
3,2023-11-02,6,665986,1.0,2008-10-01,2024-04-26,1,NaN,NaT,NaT
4,2023-11-02,6,999012,1.0,2022-10-12,2024-04-26,1,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...
14111,2024-01-20,22,665923,0.5,2008-10-01,2024-04-26,2,NaN,NaT,NaT
14112,2024-01-20,22,665926,0.5,2008-10-01,2024-04-26,2,NaN,NaT,NaT
14113,2024-01-20,22,665979,0.5,2023-07-01,2024-04-26,2,NaN,NaT,NaT
14114,2024-01-20,22,665997,0.5,2023-07-01,2024-04-26,2,Enfermedad común,2024-01-12,2024-02-17


In [19]:
# Tidy up our absence name column, and add a flag for absent
hours_associations_incidences['type_name'] = hours_associations_incidences['type_name'].fillna('')
hours_associations_incidences['was_absent'] = hours_associations_incidences.apply(lambda x: False if x['type_name'] == '' else True, axis=1)

In [20]:
# The hours table contains the work planning for every person_id, 
# but the name of the 'worked_hours' is misleading.
# We will create a new 'worked_hours' column that takes absences into account,
# and will use the current values as 'planned_hours'
hours_associations_incidences['planned_hours'] = hours_associations_incidences['worked_hours']
hours_associations_incidences['worked_hours'] = hours_associations_incidences.apply(lambda x: x['planned_hours'] if x['was_absent'] == False else 0, axis=1)
hours_associations_incidences.rename(columns={'type_name':'absence_type'},inplace=True)

In [21]:
# Adding auxiliary datetime column to consolidate the time dimension and assist BI engines
hours_associations_incidences['date_time'] = hours_associations_incidences['day'] + pd.to_timedelta(hours_associations_incidences['hour'], unit='h')

In [22]:
# Write to CSV
hours_associations_incidences.to_csv('hours_associations_incidences.csv',index=False)

In [23]:
# Write to database
schema = 'orquest_analytics'
mydb, engine = ocu.connect_db(ocu.host, ocu.user, ocu.password, ocu.dbname, schema)
mycursor = mydb.cursor()
print('Connected to database ...')
ocu.ensure_schema_exists(schema, mycursor, mydb)

hours_associations_incidences.to_sql(name='hours_associations_incidences_python',schema=schema,con=engine, if_exists='replace', index=False)

Connected to database ...


116